# Actividad 1
Navegar documentación de MLFlow y encontrar tres páginas que capten su atención.
- Explicar por qué eligieron dichas páginas.



**1. [MLflow Documentation Overview](https://mlflow.org/docs/3.2.0/ml/)**
Elegí esta página porque ofrece una visión general completa de lo que es MLflow, sus componentes principales y la manera en que se estructura la herramienta. Me pareció importante comenzar desde aquí porque ayuda a entender el panorama completo antes de entrar a ejemplos prácticos. Además, centraliza enlaces a otras secciones clave de la documentación, lo que permite tener un mapa claro de los recursos disponibles.



**2. [Tracking Quickstart](https://mlflow.org/docs/3.2.0/ml/tracking/quickstart/)**
Esta página me llamó la atención porque muestra, paso a paso, cómo comenzar a usar MLflow Tracking. Es especialmente útil para aprender a registrar experimentos y métricas de manera práctica. La elegí porque introduce código sencillo en Python que se puede correr directamente en un notebook, lo que facilita experimentar y ver resultados rápidos sin necesidad de conocimientos avanzados previos.



**3. [Logging the First Model](https://mlflow.org/docs/3.2.0/ml/getting-started/logging-first-model/)**
Decidí incluir esta página porque explica cómo guardar un modelo de machine learning dentro de MLflow por primera vez, lo cual es un paso fundamental en cualquier flujo de trabajo de ML. Es valiosa porque conecta la teoría con la práctica: no solo se ejecuta el modelo, sino que se documenta y guarda para su posterior consulta o despliegue. Esto la convierte en un recurso clave para entender la integración con scikit-learn.

---
